# 基盤人工知能演習 第3回

※本演習資料の二次配布・再配布はお断り致します。

　今回の演習の内容は以下の3つである。

**AI3.1 線形回帰 (Linear regression)**

**AI3.2 多項式回帰 (Polynomial regression)**

**AI3.3 過剰適合を防ぐ正則化（Ridge回帰）**

## AI3.1 | 線形回帰 (Linear regression)

　まず、$y = f(x_1, x_2)$ を、既知データ $X, y$ を利用することで線形回帰で推定することをおこなってみる。

### AI3.1.1 | データセットの作成

　AI3.1 で利用する仮想的なデータセットを作成する。
ここでは、$y = 3x_1 - x_2 + 2$ という線形の関係を満たすような、ノイズが無い10件のデータ $X, y$ を作っている。

In [0]:
import numpy as np
import matplotlib.pyplot as plt # 描画用（基盤データサイエンス演習 第2回でも利用）

In [0]:
# データセットの作成
n_data = 10                   # 作成するデータ数
X = np.random.rand(n_data, 2) # n_data個のxをランダムに生成
y = 3 * X[:, 0] - X[:, 1] + 2
print(X)
print(y)

　このコードで注意する点を以下にまとめておく。NumPyの使い方にも関わるので、しっかり確認しておこう。
* $X, y$ ともに10件のデータをまとめて表現している。
* $X$は、各データ$x^T=[x_1, x_2]$が**横ベクトル**として、10件縦に重なっている。（講義資料を読み返そう。確かに$X$は$x^T$を縦に重ねたものになっている）
* Pythonでは添え字は0から始まるので、10件のデータ全ての$x_1$を取得する、という操作は`X[:,0]`に対応する。
* NumPyでは、行列`X`に対して`X[:, 0]`とすると10件のデータ全ての0列目（一番左の列、すなわち $x_1$）を取得することができる。このため、 `y = 3*X[:, 0] - X[:, 1] + 2` と記述することで10件のデータの $y$ を同時に作ることができる。
  * `X[:, 0]`は**すべて（`:`）の行の、0列目のデータを取得する**という意味であると考えよう。


### AI3.1.2 | NumPyを利用した線形回帰

　通常、Pythonではscikit-learnという機械学習用のライブラリを用いて線形回帰などを行うのだが、その中身は本日講義で学んだ内容に基づいている。

　この演習は**講義で学んだものを実際に使う**ことも1つの目的であるので、ここではscikit-learnを使う前に、**線形代数の計算に基づいて** $y = w^Tx + b = w_1x_1 + w_2x_2 + b$ の $w, b$ の推定を行ってみよう。

　まず、定数項 $b$ の処理を簡単にするため、全ての要素が1である $x_3$ を`X`に追加したものを`X_aux`として定義する。これを行うことで、 $y = w^Tx = w_1x_1 + w_2x_2 + w_3x_3 = w_1x_1 + w_2x_2 + w_3$ として表現することができ、 $b$ を $w$ に含めることができる。

In [0]:
ones = np.ones((10,1)) 
X_aux = np.hstack([X, ones]) # x_3の作成
print(X_aux)

　それでは、この $w$ を10件のデータから推定しよう。講義資料によれば、$\hat w = (X^TX)^{-1}X^Ty$ を計算することで、最小二乗法による $w$ の推定が行えるので、これを計算してみよう。

In [0]:
Y = y[:, np.newaxis] # 線形代数の計算なので、縦ベクトル化しておく
print(Y)

In [0]:
def estimate_parameters(X_train, Y_train):

  XtX = np.dot(X_train.T, X_train)
  XtXinvXt = np.dot(np.linalg.inv(XtX), X_train.T)
  return np.dot(XtXinvXt, Y_train)

w_hat = estimate_parameters(X_aux, Y)
print(w_hat)

　$\hat w = [3, -1, 2]^T$ という結果が得られ、正しく $y = 3x_1 - x_2 + 2$ を推定できていることがわかる。

　今回はデータにノイズが存在せず、$x$と$y$の関係が線形なので、完全に予測することができている。

------
#### 課題 AI3.1

　算出された $\hat w$ を用いて$X = [[2, 1], [3, 2]]$ に対する $\hat y = X\hat w$ の予測を行うことを考える。
以下のコードの `__xxxxx__` 部を埋めて、 $\hat y$ を算出せよ（今回は $\hat w$ が誤差なく推定出来ているので、$\hat y$ は $y = 3x_1 - x_2 + 2$ に一致するはずである）。 

　なお、課題提出時には以下の2つを記述せよ。
* `__xxxxx__` に何を記述したか
* `print(y_hat)` の出力結果（Pythonが**整数値**として処理している場合は整数値を、**実数値**として処理している場合は小数点以下第1位まで答えよ）

In [0]:
w_hat = np.array([[3, -1, 2]]).T 
new_X = np.array([[2,1], [3,2]])           # 2つのデータ[2,1]と[3,2] を予測する 
new_X = np.hstack([new_X, np.ones((2,1))]) # new_Xにもx_3を追加してあげる
print(new_X)
y_hat = np.__xxxxx__(new_X, w_hat)
print(y_hat)

---------

### AI3.1.3 | scikit-learnを利用した線形回帰

　次に、同じ計算をscikit-learnを用いて実行してみる。


　scikit-learnで用意されている関数は極めて賢く作られており、定数項の推定のための変数の追加などは内部で実行される。

　scikit-learnで学習を行う際には、以下の順番で計算を行う。
1.  モデル（今回は線形回帰モデル）を定義する。
2.   `fit()` を実行する。
3.   必要に応じて `model.coef_` や `model.intercept_` を使って各説明変数に対する重み、および定数項を確認する。

以下で実際にやってみよう。


In [0]:
from sklearn.linear_model import LinearRegression 

# 1. 線形回帰を行うモデルの定義
# y = w^t x + bという数式だけ準備したような状態
model = LinearRegression()

# 2. fit()の実行、内部的にはAI3.1.2の計算を行っている
# Xは定数項を追加せずに入力する
# yはベクトル化せず、1次元配列として入力する
model.fit(X, y) 

# 3. 重みの確認
print(model.coef_)      # w_1, w_2 の値を確認
print(model.intercept_) # b (w_3) の値を確認

　`model.fit()` に入れる`X`, `y`はそれぞれ行列と1次元配列であることに注意してほしい。printされた結果を見てみると、（ほぼ）同じ結果が得られたことが確認できる。

　次に、このモデルを使って新しいデータ $(x_1, x_2) = (10, 30), (2, 1)$ を予測してみよう。新しいデータへの予測は `predict()` メソッドを利用する。
$y = 3x_1 - x_2 + 2$ なので、この2つはそれぞれ $y = 2, 7$ となるはずである。

In [0]:
# Xと同様に複数のデータをまとめた行列を作る
X_new = np.array([[10, 30],
                  [2, 1]])
model.predict(X_new) # 予測結果も1次元配列として出てくるので注意

　これで、NumPyで実装した場合と同様に新しいデータに対する予測を行うことができた。

## AI3.2 | 多項式回帰 (Polynomial regression) 
　次に、$y = \sin(x)$ を多項式 $\hat f(x) = w_0 + \sum_{i=1}^K w_i x^i$ で近似してみることを考える。

### AI3.2.1 | データセットの作成

　まずは、先ほどと同様に仮想的なデータを10個、ランダムに作成する。
ただし、今回のデータはAI3.1の線形回帰のデータとは異なり、 $y$ の観測誤差が平均 0.1 程度含まれているものとする。


In [0]:
# import packages
import numpy as np
import matplotlib.pyplot as plt # 描画用（基盤データサイエンス演習 第2回でも利用）

In [0]:
# create toy dataset
np.random.seed(7) # 毎回同じランダムデータが作成できるようにするおまじない

n_data = 10
x = 6 * np.random.rand(n_data) - 3         # -3~3の値を10個ランダムに生成
noise = 0.1 * np.random.randn(n_data)      # ノイズ
y = np.sin(x) + noise                      # y = sin(x) + noise を計算

　作成された **$x, y$ はどちらも1次元配列である**ことに注意しよう。作成したデータをプロットしてみる。

In [0]:
# y = sin(x) のグラフをなめらかに描画するために、
# 0.01刻みでsinの値を計算して、
# 折れ線グラフで表示する
xg = np.arange(-3, 3, 0.01)
yg = np.sin(xg)
plt.plot(xg, yg, "red", label="ground truth")

# 作成したデータを散布図として表示する
plt.scatter(x, y, label="observed data")

plt.legend(loc = "lower right") # 凡例の表示
plt.show()

　基本的には $y = \sin(x)$ に従っているが、 $y$ に関する観測誤差のためにわずかに上下に値がずれている。

### AI3.2.2 | scikit-learnを利用した多項式回帰
　それでは、多項式回帰を行うことで、 $\sin(x)$ を3次関数 $\hat f(x) = b + w_1x^1 + w_2x^2 + w_1x^3$ で近似してみる。ここで **$[s, t, u] = [x^1, x^2, x^3]$** という3つの説明変数を考えると、**3変数の線形回帰と全く同じ式**に帰着することができる（**図 AI3.1**）。

![図 AI3.1](https://i.imgur.com/zNfccQt.png)

**図 AI3.1 | 線形回帰と多項式回帰の関係**  定数項は $b$ で表現している。


　このことから、scikit-learnでは、以下の手順で3次多項式の回帰を実現する。

1. `PolynomialFeatures()` クラスを利用して スカラー値 $x$ を $3$ 個の説明変数 $[x_1, x_2, x_3]^T = [x^1, x^2, x^3]^T$ に拡張する。
2. $[x_1, x_2, x_3]^T$ を入力として、線形回帰 `LinearRegression()` を行うことで、 $[w_1, w_2, w_3]^T$ および $b$ を推定する。


　それでは早速 `PolynomialFeatures()` を使ってみよう。
`PolynomialFeatures()` 自体も入力として行列$X$を受け取るものになっている。AI3.2.1で作成した $x$ は行列になっていないので、行列表現に変換してから、 `PolynomialFeatures()` を利用する。

In [0]:
print(x)  # 元々のデータは1次元配列

In [0]:
X = x[:, np.newaxis] # N件の、1つの特徴量からなるデータ行列に変換する
print(X) 

In [0]:
from sklearn.preprocessing import PolynomialFeatures

K = 3 # 3次の項まで使う

# fit_transform()を行うことで変換ができる
# LinearRegressionで定数項（x^0の部分）は勝手に考慮されるので、include_bias=Falseにしている
# include_bias=Trueだと、x^0 = 1が各データに追加される
X_poly = PolynomialFeatures(degree=K, include_bias=False).fit_transform(X)
print(X_poly) # x^1, x^2, x^3の順番であることに注意

　なお、`2.20e+01`とは、$2.20\times 10^{1} = 22.0$ という意味である。あとは、これを使って `LinearRegression()` をすれば、多項式回帰が行えるはずである。

------
#### 課題 AI3.2

　上記の特徴量を用いて3.1節と同様に線形回帰を行うことで、多項式回帰が実現される。
AI3.1.3で実行した線形回帰のコードを参考に、`LinearRegression`モデルを用いて回帰を行い、`model.coef_`と`model.intercept_`を出力せよ。

　なお、課題提出時には、`model.coef_`と`model.intercept_`の結果を、**小数点以下第4桁を四捨五入した値**をレポートに記述せよ。

-------
#### 課題 AI3.3

　課題 AI3.2 の結果を解釈しよう。推定された3次関数式 $\hat f(x)$ を答えよ。ただし、課題 AI3.2 と同様に、係数は全て**小数点以下第4位を四捨五入し**、`f(x) = ax^3 + bx^2 + cx + d` の形式でレポートに記述せよ。

-------

### AI3.2.3 | `make_pipeline()` を利用したモデルの構築

　今回の予測は、「多項式の特徴量作成」と「線形回帰」という複数のステップを踏んだ。このような場合、いちいち途中結果を出力することなく、複数のステップを1つにまとめたモデルを作る`make_pipeline()`が利用可能である。ただし、`make_pipeline()`を利用すると重みを取り出すためにひと手間必要になるので注意が必要だ。

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

K = 3 # 3次の項まで使う

# 処理の順番通りにmake_pipeline(1つめ, 2つめ, ...)と入力する。
model_polyreg = make_pipeline(PolynomialFeatures(degree=K, include_bias=False), 
                              LinearRegression())

# 複数の処理をまとめてやるのも fit() だけでよい
X = x[:, np.newaxis] 
model_polyreg.fit(X, y)

In [0]:
# LinearRegressionはPipelineの2番目の要素なので、
# model_polyreg[1] として LinearRegression の中身にアクセスする
print(model_polyreg[1].coef_)
print(model_polyreg[1].intercept_)

#### AI3.2.4 結果の描画とパラメータ $K$ の調整

　ところで予測された $\hat f(x)$ はどれほど $\sin(x)$ に似ているだろうか。グラフを描画して確認してみる。

In [0]:
# 全ての描画を行う関数
# x, y は1次元配列
def draw(x, y, model):
  # 学習に利用したデータ点の散布図
  plt.scatter(x, y, label="observed data")

  # sin(x)の描画
  xg = np.arange(-3, 3, 0.01)
  yg = np.sin(xg)
  plt.plot(xg, yg, "red", label="ground truth")

  # 推定されたf(x)の描画
  Xg = xg[:, np.newaxis] # 行列化
  y_est = model.predict(Xg)
  plt.plot(xg, y_est, "blue", label="estimation")

  # 表示処理
  plt.legend(loc = "lower right") # 凡例の表示
  plt.show()

In [0]:
# 最初に作成したデータx, yとpipelineでつないだモデルを入力する
draw(x, y, model_polyreg)

　本来の関数が赤いグラフ、今回推定した $\hat f(x)$ が青いグラフとなっている。かなり良い近似が得られているようだ。

　さて、描画結果を見ることができるようになり、予測結果の良しあしを考えることができるようになったので、近似する多項式の次数 $K$ の値を変更させてみて、どのように関数形状が変化するか確認してみる。$K = 1$ から $K=9$ まで変更させながら、関数の形状のかみ合い具合を確認してみよ。

In [0]:
K = 6 # これが多項式の次数。1から9まで変化させてみよ

model_polyreg = make_pipeline(PolynomialFeatures(degree=K, include_bias=False), 
                              LinearRegression())
X = x[:, np.newaxis] 
model_polyreg.fit(X, y)
draw(x, y, model_polyreg)

　多項式の次数 $K$ が増えるに従い観測データ (observed data) と推定された関数 (estimation) のズレが少なくなり、真の関数 (ground truth) と形状も似てくるのだが、$K=5$ あたりから推定された関数の形状が $\sin(x)$ とかみ合わなくなり始め、 $K=6$ 以降では全く関数形状が推定できなくなってしまう。

　$K=6$の場合、3次多項式の要素 $x^1, x^2, x^3$ は全て含まれており、3次多項式と同程度以上の高精度な予測が期待できるはずなのだが、$x^4, x^5, x^6$の情報を使って観測データをより詳細に、**観測誤差までも**予測しようとし、結果として元の関数の形状を推定できなくなっている。これは**過学習 (over fitting)** と呼ばれる現象であり、**データ数が少なく、説明変数が多い時**に特に発生しやすい（**補足資料 ※1**）。

## AI3.3 | 過学習を防ぐ正則化（Ridge回帰; Ridge regression）

　AI3.2で、多項式の次元数 $K$ が非常に大きい時、多項式回帰の結果は過学習していることが分かった。
これを避ける工夫として広く用いられているのが **Ridge回帰**である。

### AI3.3.1 | scikit-learnの`Ridge()`を利用したRidge回帰の実施

　Ridge回帰はscikit-learnに`Ridge()`として定義されているので、これを利用して過学習を抑えてみる。

　`Ridge()`には正則化項の重み `alpha` をパラメータとして与える必要がある。`alpha`の値が大きいほど、過学習を抑えることができるはずだ。

　以下のコードの`alpha = 0.1`の部分を様々な値に変更させながら、推定された関数形状を確認してみよ。

In [0]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline

K = 6
alpha = 0.1

model_poly_ridge = make_pipeline(PolynomialFeatures(degree=K, include_bias=False), 
                                 Ridge(alpha=alpha))
X = x[:, np.newaxis] 
model_poly_ridge.fit(X, y)
draw(x, y, model_poly_ridge)

　どうだろうか。正しく機能しているだろうか。**実はこれだけではなかなかうまくいかない**。

　Ridge回帰の式を再度考えてみよう。 $\lambda |w|_2^2$ という項がモデルの複雑さを制御する正則化項 (l2 norm regularization) であった。この式から、正則化項の重み $\lambda$ （sklearnでは`alpha`）を各重み **$w_i$ に対して均等に**効かせる。

　一方、値の幅が大きい特徴量（今回の場合、$x^1$ よりも $x^6$ の方が値の幅（＝分散）が大きくなっているはずである）の重み $w$ は一般に小さくなるので、正則化項の重み $\lambda$ (`alpha`) の効き目が弱くなってしまうのである（**補足資料 ※2**）。

　このように、**値の幅が異なる説明変数に対してRidge回帰を適用する場合には、特徴量の値を平均0、分散1にそろえる標準化 (Standardization) を行う**と良い。標準化を行う場合は、`make_pipeline`の`Ridge`の前に`StandardScaler()`を導入する。

In [0]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline

K = 6
alpha  = 0.1

model_poly_ridge = make_pipeline(PolynomialFeatures(degree=K, include_bias=False), 
                                 StandardScaler(),
                                 Ridge(alpha=alpha))
model_poly_ridge.fit(X, y)

draw(x, y, model_poly_ridge)

　これを行うことで、高次の項まで利用した場合でも、だいぶ本来の $y = \sin(x)$ に近い関数形状を推定することができた。このように、Ridge回帰は過学習を抑えることができる。

　ただしその一方で、この予測された関数は**3次多項式による近似に比べると関数形状の推定が僅かに悪い**ことには注意する必要がある。**予測すべき対象の関数の概形が既知**で、それに対して**適切な関数（モデル）が理論や人間の感覚から推定できる**のであれば、**それより複雑なモデルを導入しても通常良い結果はもたらさない**ということは、覚えておくと良いだろう。（**※3**）

-----------
#### 課題 AI3.4

　上記の資料では、Ridge回帰の `alpha` を0.1としていた。この値を $\alpha = 1$ や $\alpha = 10$ と大きくしていった場合にestimationのグラフはどう変化するだろうか。`StandardScaler()`を含めたRidge回帰パイプラインに対して様々な$\alpha$を適用し、簡潔に解答せよ。


----

#### 課題 AI3.5

　課題 AI3.4 で示したように、 $\alpha$ の値は予測結果に大きな影響を与え、かつ最適な $\alpha$ の値はその時々によって変化してしまう。
そこで、基盤データサイエンス演習 第3回で学習した `GridSearchCV()` を用いて、**5-fold 交差検証法 (cross validation)** による $\alpha \in [10^{-4}, 10^{-3}, ..., 10^{0}, 10^1]$ のハイパーパラメータ探索を行い、この回帰問題における最適な $\alpha$ の値を推定せよ。（ヒント：基盤データサイエンス演習 第3回の資料を参考にするとよい）

　なお、pipelineの中のモデルに対して`GridSearchCV()`を行うのは簡単ではないため（不可能ではない）、以下のコードを参考に、Ridge回帰部分のみに対して`GridSearchCV()`を実施せよ。

In [0]:
# 不完全なコード
# 基盤データサイエンス演習 第3回の資料を参考に以下のコードを埋めよ

from sklearn.model_selection import GridSearchCV

K = 6
param_grid = _____________
X_poly = PolynomialFeatures(degree=K, include_bias=False).fit_transform(X)
X_poly_standardized = StandardScaler().fit_transform(X_poly)

grid_search_ridge = GridSearchCV(__________)
grid_search_ridge.fit(X_poly_standardized, y)

-------------------

#### 課題 AI3.6（発展、提出対象ではありません）

　講義資料を参考に、NumPyを用いて**Ridge回帰**と**特徴量の標準化**を実装し、予測されたグラフがほぼ同一になることを確認せよ。

-------

# レポート提出について



## レポートの提出方法

　レポートは**答案テンプレートを用い**、**1つのファイル (.doc, .docx, .pdf)** にまとめ、**学籍番号と氏名を確認の上**、**12/26 15:00 (次回 基盤人工知能演習) までに東工大ポータルのOCW-iから提出**すること。
ファイルのアップロード後、OCW-iで「提出済」というアイコンが表示されていることを必ず確認すること。それ以外の場合は未提出扱いとなるので十分注意すること。
また、締め切りを過ぎるとファイルの提出ができないため、時間に余裕を持って提出を行うこと。


## 答案テンプレート

```
学籍番号:
名前:

課題 AI3.1
__xxxxx__ = __________________
y_hat     = __________________

課題 AI3.2
model.coef_      = __________________
model.intercept_ = __________________

課題 AI3.3
f(x) = _____x^3 + _____x^2 + _____x + _____
（適宜符号を書き換えよ）

課題 AI3.4
（自由記述）

課題 AI3.5
alpha = 

```



# 補足資料



## ※1 サンプルサイズが大きければ過学習は起きにくい

　今回の資料では、サンプルサイズが10件のサンプル（データセット）からsin(x)の形状を推定することを行い、6次多項式など、高次多項式を利用すると過学習が発生することを確認した。

　その説明の中で、「**データ数が少なく**、説明変数が多い時に過学習 (over fitting) が発生しやすい」と記載されていたものの、データ数を増やした場合の評価を行うことなく、Ridge回帰に話が進んでしまった。そこで、補足資料として**データ数が10件ではなく1000件**であった時に、6次多項式を用いてみる。

　以下のコードを実行してみると、データ数が1000件もあると、Ridge回帰を導入するまでもなく、**3次多項式よりも6次多項式の方がよい関数推定を行うことができる**。モデルの柔軟性（多項式回帰なら次元数 $K$ ）は、データ数と相談しながら決定するのが望ましそうだ。

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import numpy as np

In [0]:
# create toy dataset
np.random.seed(3) # 毎回同じランダムデータが作成できるようにするおまじない

n_data = 1000
x = 6 * np.random.rand(n_data) - 3         # -3~3の値を1000個ランダムに生成
noise = 0.1 * np.random.randn(n_data)      # ノイズ
y = np.sin(x) + noise                      # y = sin(x) + noise を計算

In [0]:
K = 6 # 6次の項まで使う

# 処理の順番通りにmake_pipeline(1つめ, 2つめ, ...)と入力する。
model_polyreg = make_pipeline(PolynomialFeatures(degree=K, include_bias=False), 
                              LinearRegression())

# 複数の処理をまとめてやるのも fit() だけでよい
X = x[:, np.newaxis] 
model_polyreg.fit(X, y)
draw(x, y, model_polyreg)

　実応用時には人間の行動など、関数形状が予測困難なことが多く、結局Ridge回帰などの正則化を使うことが多いが、**簡単にデータ数を増やせるのであれば、それによって予測精度を改善できることがほとんど**である（簡単にデータを取ることができないから予測したい、という欲求が生まれる訳だが…）。

## ※2 標準化が必要な理由

　身長と体重から何らかの値をRidge回帰を使って予測することを考える。この時、身長を[m]で表現するか、[cm]で表現するかによって、身長に対する重み $w_h$ は、100倍変化するはずである。

　一方、Ridge回帰は以下の誤差関数を最小化する。

$\begin{eqnarray}\frac{1}{n} \sum^n_{i=1} l(f(x_i),y_i) + \lambda||w||^2_2\end{eqnarray}$

正則化項 $\lambda||w||^2_2$ を見ると、$w$ が100倍大きい方が正則化のペナルティが大きくなるため、身長[m]に対する重みが厳しく制限され、身長[cm]を利用した場合と異なる予測がなされてしまう。

　本質的には何も差がないはずの身長の[m]と[cm]の表現で、結果を一致させるためには、説明変数を予め同じ値の幅に整えることがよく、それが標準化という操作になっているのである。


## ※3 予測対象に適したモデルの構築

　今回対象とした $\sin(x)$ は、テイラー展開を考えると、有限次数の多項式では厳密な表現は不可能である。そのため、（十分なデータがあるという仮定のもとで）多項式近似の次数を高めれば高めるほど予測精度は高まるはずである。一方で、今回学んだように次数を高めると過学習の可能性が高くなるので、多項式近似の次数はどこかで折り合いをつける必要がある。

　このような場合は、**次数を上げても予測精度がほとんど向上しない場合は、なるべく次数の低いモデルを選ぶと良い**とされる（機械学習における「オッカムの剃刀」と言われることもある）。

　本来、このことは基盤データサイエンスで学んだ AIC (Akaike's information criterion) や BIC (Bayesian information criterion) を導入して定量的に議論すべき点なのだが、Ridge回帰などの正則化項が加わると議論が難しくなるため、ここでは割愛する。
